In [1]:
import os

import pandas as pd
import numpy as np
import plotly as py

import flask
from flask_cors import CORS
import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html

app = dash.Dash(__name__)
server = app.server

In [2]:
app.layout = html.Div([
    html.Div(
        [
            dcc.Markdown(
                '''
                ### A History of Interest Rates Since the Onset of the COVID-19 Pandemic
                This interactive report is a rendition of a
                [New York Times original](https://www.nytimes.com/interactive/2015/03/19/upshot/3d-yield-curve-economic-growth.html).
                '''.replace('  ', ''),
                className='eight columns offset-by-two'
            )
        ],
        className='row',
        style={'text-align': 'center', 'margin-bottom': '15px'}
    ),
    html.Div(
        [
            html.Div(
                [
                    dcc.Slider(
                        min=0,
                        max=5,
                        value=0,
                        marks={i: ''.format(i + 1) for i in range(6)},
                        id='slider'
                    ),
                ],
                className='row',
                style={'margin-bottom': '10px'}
            ),
            html.Div(
                [
                    html.Div(
                        [
                            html.Button('Back', id='back', style={
                                        'display': 'inline-block'}),
                            html.Button('Next', id='next', style={
                                        'display': 'inline-block'})
                        ],
                        className='two columns offset-by-two'
                    ),
                    dcc.Markdown(
                        id='text',
                        className='six columns'
                    ),
                ],
                className='row',
                style={'margin-bottom': '10px'}
            ),
            dcc.Graph(
                id='graph',
                style={'height': '60vh'}
            ),
        ],
        id='page'
    ),
])


# Internal logic
last_back = 0
last_next = 0

df = pd.read_csv("data/yield_curve.csv")

xlist = list(df["x"].dropna())
ylist = list(df["y"].dropna())

del df["x"]
del df["y"]

zlist = []
for row in df.iterrows():
    index, data = row
    zlist.append(data.tolist())

UPS = {
    0: dict(x=0, y=0, z=1),
    1: dict(x=0, y=0, z=1),
    2: dict(x=0, y=0, z=1),
    3: dict(x=0, y=0, z=1),
    4: dict(x=0, y=0, z=1),
    5: dict(x=0, y=0, z=1),
}

CENTERS = {
    0: dict(x=0.3, y=0.8, z=-0.5),
    1: dict(x=0, y=0, z=-0.37),
    2: dict(x=0, y=1.1, z=-1.3),
    3: dict(x=0, y=-0.7, z=0),
    4: dict(x=0, y=-0.2, z=0),
    5: dict(x=-0.11, y=-0.5, z=0),
}

EYES = {
    0: dict(x=2.7, y=2.7, z=0.3),
    1: dict(x=0.01, y=3.8, z=-0.37),
    2: dict(x=1.3, y=3, z=0),
    3: dict(x=2.6, y=-1.6, z=0),
    4: dict(x=3, y=-0.2, z=0),
    5: dict(x=-0.1, y=-0.5, z=2.66)
}

TEXTS = {
    0: '''
    #### Yield curve 101
    The yield curve shows how much it costs the federal government to borrow
    money for a given amount of time, revealing the relationship between long-
    and short-term interest rates.
    >>
    It is, inherently, a forecast for what the economy holds in the future —
    how much inflation there will be, for example, and how healthy growth will
    be over the years ahead — all embodied in the price of money today,
    tomorrow and many years from now.
    '''.replace('  ', ''),
    1: '''
    #### Where we stand
    On Wednesday, both short-term and long-term rates were lower than they have
    been for most of history – a reflection of the continuing hangover
    from the financial crisis.
    >>
    The yield curve is fairly flat, which is a sign that investors expect
    mediocre growth in the years ahead.
    '''.replace('  ', ''),
    2: '''
    #### Deep in the valley
    In response to the last recession, the Federal Reserve has kept short-term
    rates very low — near zero — since 2008. (Lower interest rates stimulate
    the economy, by making it cheaper for people to borrow money, but also
    spark inflation.)
    >>
    Now, the Fed is getting ready to raise rates again, possibly as early as
    June.
    '''.replace('  ', ''),
    3: '''
    #### Last time, a puzzle
    The last time the Fed started raising rates was in 2004. From 2004 to 2006,
    short-term rates rose steadily.
    >>
    But long-term rates didn't rise very much.
    >>
    The Federal Reserve chairman called this phenomenon a “conundrum," and it
    raised questions about the ability of the Fed to guide the economy.
    Part of the reason long-term rates failed to rise was because of strong
    foreign demand.
    '''.replace('  ', ''),
    4: '''
    #### Long-term rates are low now, too
    Foreign buyers have helped keep long-term rates low recently, too — as have
    new rules encouraging banks to hold government debt and expectations that
    economic growth could be weak for a long time.
    >>
    The 10-year Treasury yield was as low as it has ever been in July 2012 and
    has risen only modestly since.
    Some economists refer to the economic pessimism as “the new normal.”
    '''.replace('  ', ''),
    5: '''
    #### Long-term rates are low now, too
    Here is the same chart viewed from above.
    '''.replace('  ', '')
}

ANNOTATIONS = {
    0: [],
    1: [dict(
        showarrow=False,
        x="1-month",
        y='2015-03-18',
        z=0.046,
        text="Short-term rates basically <br>follow the interest rates set <br>by the Federal Reserve.",
        xref='x',
        yref='y',
        zref='z',
        xanchor='left',
        yanchor='auto'
    )],
    2: [],
    3: [],
    4: [],
    5: [],
}


# Make 3d graph
@app.callback(Output('graph', 'figure'), [Input('slider', 'value')])
def make_graph(value):

    if value is None:
        value = 0

    if value in [0, 2, 3]:
        z_secondary_beginning = [z[1] for z in zlist if z[0] == 'None']
        z_secondary_end = [z[0] for z in zlist if z[0] != 'None']
        z_secondary = z_secondary_beginning + z_secondary_end
        x_secondary = [
            '3-month'] * len(z_secondary_beginning) + ['1-month'] * len(z_secondary_end)
        y_secondary = ylist
        opacity = 0.7

    elif value == 1:
        x_secondary = xlist
        y_secondary = [ylist[-1] for i in xlist]
        z_secondary = zlist[-1]
        opacity = 0.7

    elif value == 4:
        z_secondary = [z[8] for z in zlist]
        x_secondary = ['10-year' for i in z_secondary]
        y_secondary = ylist
        opacity = 0.25

    if value in range(0, 5):

        trace1 = dict(
            type="surface",
            x=xlist,
            y=ylist,
            z=zlist,
            hoverinfo='x+y+z',
            lighting={
                "ambient": 0.95,
                "diffuse": 0.99,
                "fresnel": 0.01,
                "roughness": 0.01,
                "specular": 0.01,
            },
            colorscale=[[0, "rgb(230,245,254)"], [0.4, "rgb(123,171,203)"], [
                0.8, "rgb(40,119,174)"], [1, "rgb(37,61,81)"]],
            opacity=opacity,
            showscale=False,
            zmax=9.18,
            zmin=0,
            scene="scene",
        )

        trace2 = dict(
            type='scatter3d',
            mode='lines',
            x=x_secondary,
            y=y_secondary,
            z=z_secondary,
            hoverinfo='x+y+z',
            line=dict(color='#444444')
        )

        data = [trace1, trace2]

    else:

        trace1 = dict(
            type="contour",
            x=ylist,
            y=xlist,
            z=np.array(zlist).T,
            colorscale=[[0, "rgb(230,245,254)"], [0.4, "rgb(123,171,203)"], [
                0.8, "rgb(40,119,174)"], [1, "rgb(37,61,81)"]],
            showscale=False,
            zmax=9.18,
            zmin=0,
            line=dict(smoothing=1, color='rgba(40,40,40,0.15)'),
            contours=dict(coloring='heatmap')
        )

        data = [trace1]

        # margin = dict(
        #     t=5,
        #     l=50,
        #     b=50,
        #     r=5,
        # ),

    layout = dict(
        autosize=True,
        font=dict(
            size=12,
            color="#CCCCCC",
        ),
        margin=dict(
            t=5,
            l=5,
            b=5,
            r=5,
        ),
        showlegend=False,
        hovermode='closest',
        scene=dict(
            aspectmode="manual",
            aspectratio=dict(x=2, y=5, z=1.5),
            camera=dict(
                up=UPS[value],
                center=CENTERS[value],
                eye=EYES[value]
            ),
            annotations=[dict(
                showarrow=False,
                y="2015-03-18",
                x="1-month",
                z=0.046,
                text="Point 1",
                xanchor="left",
                xshift=10,
                opacity=0.7
            ), dict(
                y="2015-03-18",
                x="3-month",
                z=0.048,
                text="Point 2",
                textangle=0,
                ax=0,
                ay=-75,
                font=dict(
                    color="black",
                    size=12
                ),
                arrowcolor="black",
                arrowsize=3,
                arrowwidth=1,
                arrowhead=1
            )],
            xaxis={
                "showgrid": True,
                "title": "",
                "type": "category",
                "zeroline": False,
                "categoryorder": 'array',
                "categoryarray": list(reversed(xlist))
            },
            yaxis={
                "showgrid": True,
                "title": "",
                "type": "date",
                "zeroline": False,
            },
        )
    )

    figure = dict(data=data, layout=layout)
    # py.iplot(figure)
    return figure


# Make annotations
@app.callback(Output('text', 'children'), [Input('slider', 'value')])
def make_text(value):
    if value is None:
        value = 0

    return TEXTS[value]


# Button controls
@app.callback(Output('slider', 'value'),
              [Input('back', 'n_clicks'), Input('next', 'n_clicks')],
              [State('slider', 'value')])
def advance_slider(back, nxt, slider):

    if back is None:
        back = 0
    if nxt is None:
        nxt = 0
    if slider is None:
        slider = 0

    global last_back
    global last_next

    if back > last_back:
        last_back = back
        return max(0, slider - 1)
    if nxt > last_next:
        last_next = nxt
        return min(5, slider + 1)


In [3]:
app.server.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/May/2024 22:07:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /assets/stylesheet.css?m=1715887880.3629868 HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2024 22:07:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-component-suites/plotly/package_data/plotly.min.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2024 22:07:38] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -


In [3]:
df = pd.read_parquet("./data/yield_curve_historical_rates_MASTER.parquet")

In [4]:
df.head()

,Date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,2 Yr - 10 Yr,2 Yr - 30 Yr,10 Yr - 30 Yr
0,2024-05-07,5.51,5.48,5.45,5.41,5.13,4.82,4.60,4.48,4.47,4.47,4.70,4.61,0.35,0.21,-0.14
1,2024-05-06,5.51,5.48,5.45,5.42,5.12,4.82,4.64,4.48,4.48,4.49,4.73,4.64,0.33,0.18,-0.15
2,2024-05-05,5.51,5.48,5.45,5.41,5.12,4.81,4.63,4.48,4.49,4.50,4.75,4.66,0.31,0.15,-0.16
3,2024-05-04,5.51,5.48,5.45,5.41,5.12,4.81,4.63,4.48,4.49,4.50,4.75,4.66,0.31,0.15,-0.16
4,2024-05-03,5.51,5.48,5.45,5.41,5.12,4.81,4.63,4.48,4.49,4.50,4.75,4.66,0.31,0.15,-0.16


In [6]:
df = pd.read_csv("data/yield_curve.csv")
df.head(15)

,x,y,z[0],z[1],z[2],z[3],z[4],z[5],z[6],z[7],z[8],z[9],z[10]
0,1-month,1990-01-02,NaN,7.83,7.89,7.81,7.87,7.90,7.87,7.98,7.94,NaN,8.00
1,3-month,1990-01-03,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04
2,6-month,1990-01-04,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
3,1-year,1990-01-05,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
4,2-year,1990-01-08,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
5,3-year,1990-01-09,NaN,7.80,7.82,7.78,7.91,7.94,7.92,8.05,8.02,NaN,8.10
6,5-year,1990-01-10,NaN,7.75,7.78,7.77,7.91,7.95,7.92,8.00,8.03,NaN,8.11
7,7-year,1990-01-11,NaN,7.80,7.80,7.77,7.91,7.95,7.94,8.01,8.04,NaN,8.11
8,10-year,1990-01-12,NaN,7.74,7.81,7.76,7.93,7.98,7.99,8.07,8.10,NaN,8.17
9,20-year,1990-01-16,NaN,7.89,7.99,7.92,8.10,8.13,8.11,8.18,8.20,NaN,8.25
